In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import nltk
#nltk.download()
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from keras.layers import Dropout


In [ ]:
data1=pd.read_csv('training.csv', sep='\t')
data2=pd.read_csv('trial.csv', sep='\t')
y=data1['misogynous']
# print(training_data.head())
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stop_words=stopwords.words('english')
stop_words.append('imgflipcom')
stop_words.append('zip')
print(stop_words)
lemmatizer=WordNetLemmatizer()
#training data
for index,row in data1.iterrows():
  #print(row)
  filter_sentence=[]
  sentence=row['Text Transcription']
  sentence = sentence.lower()
  #print(sentence)
  sentence=re.sub(r'[^\w\s]','',sentence)#cleaning
  words=nltk.word_tokenize(sentence)
  words=[w for w in words if not w in stop_words]
  for word in words:
    filter_sentence.append(lemmatizer.lemmatize(word))
  #print(filter_sentence)
  listToStr = ' '.join([str(elem) for elem in filter_sentence])
  data1.loc[index,"Text Transcription"]=listToStr
#trail data
for index,row in data2.iterrows():
  #print(row)
    filter_sentence=[]
    sentence=row['Text Transcription']
    sentence = sentence.lower()
    #print(sentence)
    sentence=re.sub(r'[^\w\s]','',sentence)#cleaning
    words=nltk.word_tokenize(sentence)
    words=[w for w in words if not w in stop_words]
    for word in words:
        filter_sentence.append(lemmatizer.lemmatize(word))
    #print(filter_sentence)
    listToStr = ' '.join([str(elem) for elem in filter_sentence])
    data2.loc[index,"Text Transcription"]=listToStr
#print(data1['Text Transcription'].head())
#print(data1.shape)
#print(data2.head())
#print(data2.shape)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'thr

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [ ]:
def get_max_length(df):
    
#get max token counts from train data, so we use this number as fixed length input to LSTM cell
    
    max_length = 0
    for row in df['Text Transcription']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def word2vec(reviews):
#get word2vec value for each word in sentence.concatenate word in numpy array, so we can use it as LSTM input
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def padded_encoded(encoded_reviews):
#for short sentences, we prepend zero padding so all input to LSTM has same length
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
#return one hot encoding for Y value
    if sentiment == 1:
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
#encode text value to numeric value
# encode words into word2vec
    reviews = df['Text Transcription'].tolist()
    
    encoded_reviews = word2vec(reviews)
    padded_encoded_reviews = padded_encoded(encoded_reviews)
    # encoded sentiment
    sentiments = df['misogynous'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

In [ ]:
max_length = get_max_length(data1)

train_X, train_Y = preprocess(data1)

In [ ]:

model=Sequential()
model.add((LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_X,train_Y,validation_split=0.2,epochs=20,batch_size=100)

Epoch 1/20
80/80 [==============================] - 26s 294ms/step - loss: 0.6721 - accuracy: 0.5969 - val_loss: 0.6524 - val_accuracy: 0.6205
Epoch 2/20
80/80 [==============================] - 23s 290ms/step - loss: 0.6327 - accuracy: 0.6511 - val_loss: 0.6316 - val_accuracy: 0.6580
Epoch 3/20
80/80 [==============================] - 23s 291ms/step - loss: 0.6047 - accuracy: 0.6744 - val_loss: 0.6116 - val_accuracy: 0.6680
Epoch 4/20
80/80 [==============================] - 23s 284ms/step - loss: 0.5892 - accuracy: 0.6880 - val_loss: 0.6043 - val_accuracy: 0.6745
Epoch 5/20
80/80 [==============================] - 23s 288ms/step - loss: 0.5721 - accuracy: 0.6995 - val_loss: 0.5994 - val_accuracy: 0.6825
Epoch 6/20
80/80 [==============================] - 23s 288ms/step - loss: 0.5616 - accuracy: 0.7078 - val_loss: 0.5997 - val_accuracy: 0.6840
Epoch 7/20
80/80 [==============================] - 23s 290ms/step - loss: 0.5501 - accuracy: 0.7190 - val_loss: 0.6001 - val_accuracy: 0.6800

In [ ]:
test_X, test_Y = preprocess(data2)

In [ ]:
predict_x=model.predict(test_X) 
y_pred=np.argmax(predict_x,axis=1)

In [ ]:
score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

4/4 - 0s - loss: 0.4617 - accuracy: 0.8000 - 125ms/epoch - 31ms/step
Test score: 0.46173787117004395
Test accuracy: 0.800000011920929
